<a href="https://colab.research.google.com/github/init5god/init5god/blob/main/NFL_predictor_scores_ipynb_txt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Taken from: https://www.opensourcefootball.com/posts/2021-01-21-nfl-game-prediction-using-logistic-regression/

In [3]:
import nfl_data_py as nfl
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib.ticker as plticker
import numpy as np
import warnings
import itertools
warnings.simplefilter(action='ignore', category=FutureWarning)

from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

plt.style.use('seaborn-talk')
plt.style.use('ggplot')

pd.set_option('display.max_columns', 20)

<ipython-input-3-c3cdc236c14b>:13: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-talk')


In [2]:
!pip install nfl_data_py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 39.2 MB/s eta 0:00:00


# Continuamos para extraer todos los datos desde 1999 hasta la temporada actual

In [4]:
data = pd.concat([nfl.import_pbp_data([season]).assign(season=season) for season in range(1999, 2024)])

1999 done.
Downcasting floats.
2000 done.
Downcasting floats.
2001 done.
Downcasting floats.
2002 done.
Downcasting floats.
2003 done.
Downcasting floats.
2004 done.
Downcasting floats.
2005 done.
Downcasting floats.
2006 done.
Downcasting floats.
2007 done.
Downcasting floats.
2008 done.
Downcasting floats.
2009 done.
Downcasting floats.
2010 done.
Downcasting floats.
2011 done.
Downcasting floats.
2012 done.
Downcasting floats.
2013 done.
Downcasting floats.
2014 done.
Downcasting floats.
2015 done.
Downcasting floats.
2016 done.
Downcasting floats.
2017 done.
Downcasting floats.
2018 done.
Downcasting floats.
2019 done.
Downcasting floats.
2020 done.
Downcasting floats.
2021 done.
Downcasting floats.
2022 done.
Downcasting floats.
2023 done.
Downcasting floats.


# Continua programa

In [5]:
def dynamic_window_ewma(x):
    """
    Calculate rolling exponentially weighted EPA with a dynamic window size
    """
    values = np.zeros(len(x))
    for i, (_, row) in enumerate(x.iterrows()):
        epa = x.epa_shifted[:i+1]
        if row.week > 10:
            values[i] = epa.ewm(min_periods=1, span=row.week).mean().values[-1]
        else:
            values[i] = epa.ewm(min_periods=1, span=10).mean().values[-1]

    return pd.Series(values, index=x.index)

# seperate EPA in to rushing offense, rushing defense, passing offense, passing defense for each team
rushing_offense_epa = data.loc[data['rush_attempt'] == 1, :]\
.groupby(['posteam', 'season', 'week'], as_index=False)['epa'].mean()

rushing_defense_epa = data.loc[data['rush_attempt'] == 1, :]\
.groupby(['defteam', 'season', 'week'], as_index=False)['epa'].mean()

passing_offense_epa = data.loc[data['pass_attempt'] == 1, :]\
.groupby(['posteam', 'season', 'week'], as_index=False)['epa'].mean()

passing_defense_epa = data.loc[data['pass_attempt'] == 1, :]\
.groupby(['defteam', 'season', 'week'], as_index=False)['epa'].mean()

# lag EPA one period back
rushing_offense_epa['epa_shifted'] = rushing_offense_epa.groupby('posteam')['epa'].shift()
rushing_defense_epa['epa_shifted'] = rushing_defense_epa.groupby('defteam')['epa'].shift()
passing_offense_epa['epa_shifted'] = passing_offense_epa.groupby('posteam')['epa'].shift()
passing_defense_epa['epa_shifted'] = passing_defense_epa.groupby('defteam')['epa'].shift()

# In each case, calculate EWMA with a static window and dynamic window and assign it as a column
# rushing_offense_epa['ewma'] = rushing_offense_epa.groupby('posteam')['epa_shifted']\
# .transform(lambda x: x.ewm(min_periods=1, span=10).mean())

rushing_offense_epa['ewma_dynamic_window'] = rushing_offense_epa.groupby('posteam')\
.apply(dynamic_window_ewma).values

# rushing_defense_epa['ewma'] = rushing_defense_epa.groupby('defteam')['epa_shifted']\
# .transform(lambda x: x.ewm(min_periods=1, span=10).mean())

rushing_defense_epa['ewma_dynamic_window'] = rushing_defense_epa.groupby('defteam')\
.apply(dynamic_window_ewma).values

# passing_offense_epa['ewma'] = passing_offense_epa.groupby('posteam')['epa_shifted']\
# .transform(lambda x: x.ewm(min_periods=1, span=10).mean())

passing_offense_epa['ewma_dynamic_window'] = passing_offense_epa.groupby('posteam')\
.apply(dynamic_window_ewma).values

# passing_defense_epa['ewma'] = passing_defense_epa.groupby('defteam')['epa_shifted']\
# .transform(lambda x: x.ewm(min_periods=1, span=10).mean())

passing_defense_epa['ewma_dynamic_window'] = passing_defense_epa.groupby('defteam')\
.apply(dynamic_window_ewma).values

#Merge all the data together
offense_epa = rushing_offense_epa.merge(passing_offense_epa, on=['posteam', 'season', 'week'], suffixes=('_rushing', '_passing'))\
.rename(columns={'posteam': 'team'})
defense_epa = rushing_defense_epa.merge(passing_defense_epa, on=['defteam', 'season', 'week'], suffixes=('_rushing', '_passing'))\
.rename(columns={'defteam': 'team'})
epa = offense_epa.merge(defense_epa, on=['team', 'season', 'week'], suffixes=('_offense', '_defense'))

#remove the first season of data
epa = epa.loc[epa['season'] != epa['season'].unique()[0], :]

epa = epa.reset_index(drop=True)

epa.head()

,team,season,week,epa_rushing_offense,epa_shifted_rushing_offense,ewma_dynamic_window_rushing_offense,epa_passing_offense,epa_shifted_passing_offense,ewma_dynamic_window_passing_offense,epa_rushing_defense,epa_shifted_rushing_defense,ewma_dynamic_window_rushing_defense,epa_passing_defense,epa_shifted_passing_defense,ewma_dynamic_window_passing_defense
0,ARI,2000,1,-0.345669,-0.068545,-0.109518,0.019830,0.056256,-0.172588,0.166199,0.363190,0.086016,-0.009200,0.269840,0.115978
1,ARI,2000,2,-0.276743,-0.345669,-0.153920,0.558977,0.019830,-0.136410,-0.065981,0.166199,0.101092,0.283460,-0.009200,0.092441
2,ARI,2000,4,-0.334533,-0.276743,-0.176871,-0.264054,0.558977,-0.006468,-0.018524,-0.065981,0.069872,0.500345,0.283460,0.128136
3,ARI,2000,5,-0.016838,-0.334533,-0.206184,0.049141,-0.264054,-0.054359,0.004625,-0.018524,0.053437,0.058499,0.500345,0.197339
4,ARI,2000,6,-0.038473,-0.016838,-0.171124,0.101830,0.049141,-0.035195,0.086308,0.004625,0.044399,-0.063633,0.058499,0.171631


# Continuamos con el programa

In [6]:
schedule = data[['season', 'week', 'home_team', 'away_team', 'home_score', 'away_score']]\
.drop_duplicates().reset_index(drop=True)\
.assign(home_team_win = lambda x: (x.home_score > x.away_score).astype(int))

df = schedule.merge(epa.rename(columns={'team': 'home_team'}), on=['home_team', 'season', 'week'])\
.merge(epa.rename(columns={'team': 'away_team'}), on=['away_team', 'season', 'week'], suffixes=('_home', '_away'))

df.tail()

,season,week,home_team,away_team,home_score,away_score,home_team_win,epa_rushing_offense_home,epa_shifted_rushing_offense_home,ewma_dynamic_window_rushing_offense_home,...,ewma_dynamic_window_rushing_offense_away,epa_passing_offense_away,epa_shifted_passing_offense_away,ewma_dynamic_window_passing_offense_away,epa_rushing_defense_away,epa_shifted_rushing_defense_away,ewma_dynamic_window_rushing_defense_away,epa_passing_defense_away,epa_shifted_passing_defense_away,ewma_dynamic_window_passing_defense_away
6440,2023,20,BUF,KC,24,27,0,0.138143,0.261939,0.067671,...,-0.049037,0.317385,0.196790,0.041203,0.138143,0.062528,-0.021073,0.148241,-0.340118,-0.164232
6441,2023,20,DET,TB,31,23,1,0.048534,-0.044500,-0.048893,...,-0.170045,0.083998,0.288088,0.091341,0.048534,-0.370516,-0.133125,0.180241,-0.081463,-0.007964
6442,2023,21,SF,DET,34,31,1,0.218215,0.041095,0.033895,...,-0.040821,0.162754,0.180241,0.174092,0.218215,0.016074,-0.105763,0.200745,0.083998,0.083886
6443,2023,21,BAL,KC,10,17,0,0.129910,0.258387,0.001516,...,-0.030982,0.189838,0.317385,0.070570,0.129910,0.138143,-0.005860,-0.292842,0.148241,-0.132756
6444,2023,22,KC,SF,25,22,1,-0.250888,-0.277561,-0.051869,...,0.047640,0.228612,0.200745,0.252240,-0.250888,0.078229,-0.040810,0.123697,0.162754,-0.057190


In [ ]:
#Este se deja aqui por historicidad pero no es necesario par el programa
def rolling_scores(team):
    team.rolling(3).mean()
    return team

rolling_home_score = df.groupby(['home_team','season'],as_index=False)['home_score'].rolling(3).mean().reset_index(drop=True)
df['home_score_avg']= df.groupby(['home_team','season'],as_index=False)['home_score'].transform(lambda s: s.rolling(3, min_periods=1).mean())
df['away_score_avg']= df.groupby(['away_team','season'],as_index=False)['away_score'].transform(lambda s: s.rolling(3, min_periods=1).mean())
rolling_away_score = df.groupby(['away_team','season'],as_index=False)['away_score'].rolling(3).mean()

# rolling_scores = pd.concat([rolling_home_score,rolling_away_score],axis=0)
# rolling_scores
# rolling_home_score.loc[rolling_home_score['home_team']=='ARI'].head(20)

In [ ]:
df.head()

,season,week,home_team,away_team,home_score,away_score,home_team_win,epa_rushing_offense_home,epa_shifted_rushing_offense_home,ewma_dynamic_window_rushing_offense_home,...,epa_shifted_passing_offense_away,ewma_dynamic_window_passing_offense_away,epa_rushing_defense_away,epa_shifted_rushing_defense_away,ewma_dynamic_window_rushing_defense_away,epa_passing_defense_away,epa_shifted_passing_defense_away,ewma_dynamic_window_passing_defense_away,home_score_avg,away_score_avg
0,2000,1,NYG,ARI,21,16,1,0.166199,0.162852,-0.105274,...,0.056256,-0.172588,0.166199,0.363190,0.086016,-0.009200,0.269840,0.115978,21.0,16.0
1,2000,1,PIT,BAL,0,16,0,-0.499352,-0.643877,-0.117570,...,-0.434684,-0.114439,-0.499352,-0.241927,-0.204228,-0.216112,-0.314740,-0.201766,0.0,16.0
2,2000,1,WAS,CAR,20,17,1,-0.006129,-0.390855,-0.060330,...,0.144870,0.187536,-0.006129,0.060270,0.057274,0.147979,-0.368611,0.001226,20.0,17.0
3,2000,1,MIN,CHI,30,27,1,0.283000,-0.163698,-0.066504,...,-0.446245,-0.079708,0.283000,-0.130394,-0.057451,0.124496,0.365886,0.106318,30.0,27.0
4,2000,1,LA,DEN,41,36,1,-0.106372,-0.633393,-0.247472,...,-0.142305,-0.017070,-0.106372,-0.110182,-0.091425,0.400226,-0.078273,-0.086481,41.0,36.0


In [7]:
removed_columns = list(df.columns[:7])
selected_columns = df.columns[~df.columns.isin(removed_columns)]
selected_columns

Index(['epa_rushing_offense_home', 'epa_shifted_rushing_offense_home',
       'ewma_dynamic_window_rushing_offense_home', 'epa_passing_offense_home',
       'epa_shifted_passing_offense_home',
       'ewma_dynamic_window_passing_offense_home', 'epa_rushing_defense_home',
       'epa_shifted_rushing_defense_home',
       'ewma_dynamic_window_rushing_defense_home', 'epa_passing_defense_home',
       'epa_shifted_passing_defense_home',
       'ewma_dynamic_window_passing_defense_home', 'epa_rushing_offense_away',
       'epa_shifted_rushing_offense_away',
       'ewma_dynamic_window_rushing_offense_away', 'epa_passing_offense_away',
       'epa_shifted_passing_offense_away',
       'ewma_dynamic_window_passing_offense_away', 'epa_rushing_defense_away',
       'epa_shifted_rushing_defense_away',
       'ewma_dynamic_window_rushing_defense_away', 'epa_passing_defense_away',
       'epa_shifted_passing_defense_away',
       'ewma_dynamic_window_passing_defense_away'],
      dtype='object')

In [9]:
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
# df[['home_score_avg','away_score_avg']] = mms.fit_transform(df[['home_score_avg','away_score_avg']])
df[selected_columns] = mms.fit_transform(df[selected_columns])

In [10]:
df

,season,week,home_team,away_team,home_score,away_score,home_team_win,epa_rushing_offense_home,epa_shifted_rushing_offense_home,ewma_dynamic_window_rushing_offense_home,...,ewma_dynamic_window_rushing_offense_away,epa_passing_offense_away,epa_shifted_passing_offense_away,ewma_dynamic_window_passing_offense_away,epa_rushing_defense_away,epa_shifted_rushing_defense_away,ewma_dynamic_window_rushing_defense_away,epa_passing_defense_away,epa_shifted_passing_defense_away,ewma_dynamic_window_passing_defense_away
0,2000,1,NYG,ARI,21,16,1,0.796368,0.815520,0.572925,...,0.512500,0.617273,0.479423,0.438684,0.796368,0.879340,0.824789,0.427801,0.718305,0.622628
1,2000,1,PIT,BAL,0,16,0,0.517485,0.558527,0.555408,...,0.528922,0.634788,0.312533,0.490127,0.517485,0.686573,0.391050,0.353658,0.559299,0.300252
2,2000,1,WAS,CAR,20,17,1,0.724158,0.639130,0.636949,...,0.406033,0.586317,0.509547,0.757275,0.724158,0.782841,0.781838,0.484124,0.544646,0.506204
3,2000,1,MIN,CHI,30,27,1,0.845311,0.711494,0.628155,...,0.402381,0.642444,0.308603,0.520852,0.845311,0.722103,0.610392,0.475709,0.744429,0.612828
4,2000,1,LA,DEN,41,36,1,0.682154,0.561867,0.370358,...,0.534383,0.677656,0.411925,0.576266,0.682154,0.728542,0.559623,0.574512,0.623618,0.417218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6440,2023,20,BUF,KC,24,27,0,0.784612,0.847085,0.819292,...,0.597914,0.694098,0.527196,0.627818,0.784612,0.783560,0.664756,0.484218,0.552396,0.338334
6441,2023,20,DET,TB,31,23,1,0.747063,0.749465,0.653243,...,0.427021,0.633841,0.558232,0.672174,0.747063,0.645609,0.497306,0.495684,0.622750,0.496879
6442,2023,21,SF,DET,34,31,1,0.818164,0.776733,0.771178,...,0.609517,0.654174,0.521571,0.745381,0.818164,0.768762,0.538195,0.503032,0.667756,0.590069
6443,2023,21,BAL,KC,10,17,0,0.781162,0.845954,0.725051,...,0.623412,0.661167,0.568191,0.653798,0.781162,0.807648,0.687490,0.326163,0.685230,0.370268


In [11]:
df.isna().sum()

,0
season,0
week,0
home_team,0
away_team,0
home_score,0
away_score,0
home_team_win,0
epa_rushing_offense_home,0
epa_shifted_rushing_offense_home,1
ewma_dynamic_window_rushing_offense_home,1


In [12]:
df = df.copy()
df = df.fillna(0)

In [13]:
df.isna().sum()

,0
season,0
week,0
home_team,0
away_team,0
home_score,0
away_score,0
home_team_win,0
epa_rushing_offense_home,0
epa_shifted_rushing_offense_home,0
ewma_dynamic_window_rushing_offense_home,0


In [14]:
target = 'home_score'
target2 = 'away_score' #
features = [column for column in df.columns if 'ewma' in column and 'dynamic' in column]
for feature in features:
  print(feature)

ewma_dynamic_window_rushing_offense_home
ewma_dynamic_window_passing_offense_home
ewma_dynamic_window_rushing_defense_home
ewma_dynamic_window_passing_defense_home
ewma_dynamic_window_rushing_offense_away
ewma_dynamic_window_passing_offense_away
ewma_dynamic_window_rushing_defense_away
ewma_dynamic_window_passing_defense_away


In [15]:
from sklearn.linear_model import Ridge
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import TimeSeriesSplit

rr = Ridge(alpha=1)
rr2 = Ridge(alpha=1)

split = TimeSeriesSplit(n_splits=3)

sfs = SequentialFeatureSelector(rr,
                                n_features_to_select=20,
                                direction="forward",
                                cv=split,
                                n_jobs=8
                               )

In [16]:
def backtest(data, model, predictors, start=5, step=1):
    all_predictions = []

    years = sorted(data["season"].unique())

    for i in range(start, len(years), step):
        current_year = years[i]
        train = data[data["season"] < current_year]
        test = data[data["season"] == current_year]

        model.fit(train[predictors], train["home_score"])

        preds = model.predict(test[predictors])
        preds = pd.Series(preds, index=test.index)
        combined = pd.concat([test["home_score"], preds], axis=1)
        combined.columns = ["actual", "prediction"]

        all_predictions.append(combined)
    return pd.concat(all_predictions)

In [17]:
predictions = backtest(df, rr, features)

In [18]:
predictions

,actual,prediction
1317,42,18.793498
1318,9,21.474106
1319,13,18.540598
1320,24,27.975655
1321,34,20.745584
...,...,...
6440,24,24.616215
6441,31,25.616356
6442,34,28.445291
6443,10,25.253380


In [20]:
train = df[df["season"] < 2023]
rr.fit(train[features].values, train["home_score"])
rr2.fit(train[features], train["away_score"])

Ridge(alpha=1)

In [21]:
predictions = backtest(df, rr, features)

In [22]:
predictions

,actual,prediction
1317,42,18.793498
1318,9,21.474106
1319,13,18.540598
1320,24,27.975655
1321,34,20.745584
...,...,...
6440,24,24.616215
6441,31,25.616356
6442,34,28.445291
6443,10,25.253380


# Predictiones del dataframe por temporada

In [23]:
df_2022 = df.loc[(df['season'] == 2023)].assign(
    home_score_pred = lambda x: np.round(rr.predict(x[features])),
    away_score_pred = lambda x: np.round(rr2.predict(x[features]))
)\
[['home_team', 'away_team', 'week', 'home_score','home_score_pred','away_score','away_score_pred']]


df_2022['diff'] = df_2022.apply(lambda x: x.home_score - x.home_score_pred, axis=1)


df_2022.loc[(df_2022['week'] == 14)]


,home_team,away_team,week,home_score,home_score_pred,away_score,away_score_pred,diff
6353,KC,BUF,14,17,24.0,20,23.0,-7.0
6354,NO,CAR,14,28,24.0,6,17.0,4.0
6355,LAC,DEN,14,7,23.0,24,19.0,-16.0
6356,CHI,DET,14,28,21.0,13,22.0,7.0
6357,NYG,GB,14,24,19.0,22,24.0,5.0
6358,NYJ,HOU,14,30,17.0,6,22.0,13.0
6359,CIN,IND,14,34,24.0,14,21.0,10.0
6360,CLE,JAX,14,31,22.0,27,20.0,9.0
6361,BAL,LA,14,37,25.0,31,19.0,12.0
6362,LV,MIN,14,0,20.0,3,21.0,-20.0


In [24]:
fraom sklearn.metrics import mean_squared_error

mean_squared_error(df_2022["home_score"], df_2022["home_score_pred"])

99.96140350877192

In [25]:
df_2022['diff'].mean()

0.7403508771929824

In [26]:
def ewma(data, window):
    """
    Calculate the most recent value for EWMA given an array of data and a window size
    """
    alpha = 2 / (window + 1.0)
    alpha_rev = 1 - alpha
    scale = 1 / alpha_rev
    n = data.shape[0]
    r = np.arange(n)
    scale_arr = scale**r
    offset = data[0] * alpha_rev**(r+1)
    pw0 = alpha * alpha_rev**(n-1)
    mult = data * pw0 * scale_arr
    cumsums = mult.cumsum()
    out = offset + cumsums * scale_arr[::-1]
    return out[-1]

#  Next lines gives the new week scores


In [27]:
full_schedule = nfl.import_schedules([2024])

In [28]:
week_schedule = full_schedule.loc[(full_schedule['week'] == 1)]

In [29]:
week_schedule

,game_id,season,game_type,week,gameday,weekday,gametime,away_team,away_score,home_team,...,wind,away_qb_id,home_qb_id,away_qb_name,home_qb_name,away_coach,home_coach,referee,stadium_id,stadium
6706,2024_01_BAL_KC,2024,REG,1,2024-09-05,Thursday,20:20,BAL,20.0,KC,...,8.0,00-0034796,00-0033873,Lamar Jackson,Patrick Mahomes,John Harbaugh,Andy Reid,Shawn Hochuli,KAN00,GEHA Field at Arrowhead Stadium
6707,2024_01_GB_PHI,2024,REG,1,2024-09-06,Friday,20:15,GB,29.0,PHI,...,NaN,00-0036264,00-0036389,Jordan Love,Jalen Hurts,Matt LaFleur,Nick Sirianni,Ron Torbert,SAO00,Arena Corinthians
6708,2024_01_PIT_ATL,2024,REG,1,2024-09-08,Sunday,13:00,PIT,18.0,ATL,...,NaN,00-0036945,00-0029604,Justin Fields,Kirk Cousins,Mike Tomlin,Raheem Morris,Brad Rogers,ATL97,Mercedes-Benz Stadium
6709,2024_01_ARI_BUF,2024,REG,1,2024-09-08,Sunday,13:00,ARI,28.0,BUF,...,20.0,00-0035228,00-0034857,Kyler Murray,Josh Allen,Jonathan Gannon,Sean McDermott,Tra Blake,BUF00,New Era Field
6710,2024_01_TEN_CHI,2024,REG,1,2024-09-08,Sunday,13:00,TEN,17.0,CHI,...,8.0,00-0039152,00-0039918,Will Levis,Caleb Williams,Brian Callahan,Matt Eberflus,Shawn Smith,CHI98,Soldier Field
6711,2024_01_NE_CIN,2024,REG,1,2024-09-08,Sunday,13:00,NE,16.0,CIN,...,5.0,00-0033119,00-0036442,Jacoby Brissett,Joe Burrow,Jerod Mayo,Zac Taylor,Land Clark,CIN00,Paycor Stadium
6712,2024_01_HOU_IND,2024,REG,1,2024-09-08,Sunday,13:00,HOU,29.0,IND,...,NaN,00-0039163,00-0039164,C.J. Stroud,Anthony Richardson,DeMeco Ryans,Shane Steichen,John Hussey,IND00,Lucas Oil Stadium
6713,2024_01_JAX_MIA,2024,REG,1,2024-09-08,Sunday,13:00,JAX,17.0,MIA,...,10.0,00-0036971,00-0036212,Trevor Lawrence,Tua Tagovailoa,Doug Pederson,Mike McDaniel,Craig Wrolstad,MIA00,Hard Rock Stadium
6714,2024_01_CAR_NO,2024,REG,1,2024-09-08,Sunday,13:00,CAR,10.0,NO,...,NaN,00-0039150,00-0031280,Bryce Young,Derek Carr,Dave Canales,Dennis Allen,Alan Eck,NOR00,Mercedes-Benz Superdome
6715,2024_01_MIN_NYG,2024,REG,1,2024-09-08,Sunday,13:00,MIN,28.0,NYG,...,10.0,00-0034869,00-0035710,Sam Darnold,Daniel Jones,Kevin O'Connell,Brian Daboll,Alex Kemp,NYC01,MetLife Stadium


In [34]:
home_teams = list(week_schedule["home_team"].to_numpy())
away_teams = list(week_schedule["away_team"].to_numpy())


We create a function to make predictions based on how teams played given histoeric data

In [30]:
def predictions(model,data_2022_week,local_team, away_team):
    offense = data_2022_week.loc[(data_2022_week['posteam'] == local_team) | (data_2022_week['posteam'] == away_team)]
    defense = data_2022_week.loc[(data_2022_week['defteam'] == local_team) | (data_2022_week['defteam'] == away_team)]

    rushing_offense = offense.loc[offense['rush_attempt'] == 1]\
    .groupby(['posteam', 'week'], as_index=False)['epa'].mean().rename(columns={'posteam': 'team'})
    passing_offense = offense.loc[offense['pass_attempt'] == 1]\
    .groupby(['posteam', 'week'], as_index=False)['epa'].mean().rename(columns={'posteam': 'team'})
    rushing_defense = defense.loc[defense['rush_attempt'] == 1]\
    .groupby(['defteam', 'week'], as_index=False)['epa'].mean().rename(columns={'defteam': 'team'})
    passing_defense = defense.loc[defense['pass_attempt'] == 1]\
    .groupby(['defteam', 'week'], as_index=False)['epa'].mean().rename(columns={'defteam': 'team'})

    super_bowl_X = np.zeros(8)

    for i, (tm, stat_df) in enumerate(itertools.product([local_team, away_team], [rushing_offense, passing_offense, rushing_defense, passing_defense])):
        ewma_value = ewma(stat_df.loc[stat_df['team'] == tm]['epa'].values, 20)
        super_bowl_X[i] = ewma_value

    superbowl = pd.DataFrame(super_bowl_X)
    super_bowl = mms.fit_transform(superbowl)

    predicted_score = model.predict(super_bowl.reshape(1, 8))[0]
    # predicted_proba = model.predict_proba(super_bowl_X.reshape(1, 8))[0]

    # winner = local_team if predicted_score else away_team
    # loser = away_team if predicted_score else local_team
    # win_prob = predicted_proba[-1] if predicted_score else predicted_proba[0]

    return predicted_score
    # print(f'Model predicts {winner} will win over {loser} and {winner} has a {round(win_prob*100, 2)}% win probability')

We define how far we wanna go back in the seasons to get the historic data. And call the functions to get predictions

In [31]:
data_2022_week = data.loc[(data['season'] == 2023)]


In [35]:

home_scores_pred = []
away_scores_pred = []

for i  in range(len(home_teams)):
    # score_home = predictions(rr,data_2022_week,home_teams[i],away_teams[i])
    # score_away = predictions(rr,data_2022_week,away_teams[i],home_teams[i])
    # print(winner)
    home_scores_pred.append(predictions(rr,data_2022_week,home_teams[i],away_teams[i]))
    away_scores_pred.append(predictions(rr,data_2022_week,away_teams[i],home_teams[i]))
    # losers.append(loser)
    # win_probs.append(round(win_prob*100,2))

final_home_score = pd.Series(np.round(home_scores_pred,1))
final_away_score = pd.Series(np.round(away_scores_pred,1))
home_team_list = pd.Series(home_teams)
away_team_list = pd.Series(away_teams)

predictions_df = pd.DataFrame({ 'home_team': home_team_list, 'home_score_pred': final_home_score, 'away_team': away_team_list,'away_score_pred': final_away_score })

# pd.DataFrame([final_home_score,final_away_score],columns=['Home_Score','Away_Score'])


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but Ridge was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but Ridge was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but Ridge was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but Ridge was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but Ridge was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but Ridge was fitted with feature names
  warning

In [36]:
# week_results = week_schedule.loc[week_schedule['week']==11,['home_team','home_score','away_team','away_score']].reset_index(drop=True)
week_results = week_schedule.loc[:,['home_team','home_score','away_team','away_score']].reset_index(drop=True)

In [37]:
predictions = week_results.merge(predictions_df)
predictions[['home_team','away_team','home_score','home_score_pred','away_score','away_score_pred']]


,home_team,away_team,home_score,home_score_pred,away_score,away_score_pred
0,KC,BAL,27.0,16.9,20.0,31.9
1,PHI,GB,34.0,10.8,29.0,28.4
2,ATL,PIT,10.0,22.1,18.0,15.9
3,BUF,ARI,34.0,36.4,28.0,3.9
4,CHI,TEN,24.0,15.8,17.0,16.5
5,CIN,NE,10.0,24.3,16.0,16.9
6,IND,HOU,27.0,23.8,29.0,31.8
7,MIA,JAX,20.0,24.9,17.0,17.7
8,NO,CAR,47.0,32.9,10.0,8.4
9,NYG,MIN,6.0,13.3,28.0,26.6


In [39]:
predictions['away_diff'] = predictions['away_score'] - predictions['away_score_pred']
predictions['home_diff'] = predictions['home_score'] - predictions['home_score_pred']


In [40]:
predictions

,home_team,home_score,away_team,away_score,home_score_pred,away_score_pred,away_diff,home_diff
0,KC,27.0,BAL,20.0,16.9,31.9,-11.9,10.1
1,PHI,34.0,GB,29.0,10.8,28.4,0.6,23.2
2,ATL,10.0,PIT,18.0,22.1,15.9,2.1,-12.1
3,BUF,34.0,ARI,28.0,36.4,3.9,24.1,-2.4
4,CHI,24.0,TEN,17.0,15.8,16.5,0.5,8.2
5,CIN,10.0,NE,16.0,24.3,16.9,-0.9,-14.3
6,IND,27.0,HOU,29.0,23.8,31.8,-2.8,3.2
7,MIA,20.0,JAX,17.0,24.9,17.7,-0.7,-4.9
8,NO,47.0,CAR,10.0,32.9,8.4,1.6,14.1
9,NYG,6.0,MIN,28.0,13.3,26.6,1.4,-7.3


In [41]:
predictions[['home_team','away_team','home_score_pred','away_score_pred']]

,home_team,away_team,home_score_pred,away_score_pred
0,KC,BAL,16.9,31.9
1,PHI,GB,10.8,28.4
2,ATL,PIT,22.1,15.9
3,BUF,ARI,36.4,3.9
4,CHI,TEN,15.8,16.5
5,CIN,NE,24.3,16.9
6,IND,HOU,23.8,31.8
7,MIA,JAX,24.9,17.7
8,NO,CAR,32.9,8.4
9,NYG,MIN,13.3,26.6


In [ ]:
predictions.assign(home_team_win = lambda x: (x.home_score > x.away_score).astype(int)).assign(home_team_win_pred = lambda x: (x.home_score_pred > x.away_score_pred).astype(int))

,home_team,home_score,away_team,away_score,home_score_pred,away_score_pred,home_team_win,home_team_win_pred
0,KC,27.0,BAL,20.0,16.9,31.9,1,0
1,PHI,34.0,GB,29.0,10.8,28.4,1,0
2,ATL,10.0,PIT,18.0,22.1,15.9,0,1
3,BUF,34.0,ARI,28.0,36.4,3.9,1,1
4,CHI,24.0,TEN,17.0,15.8,16.5,1,0
5,CIN,10.0,NE,16.0,24.3,16.9,0,1
6,IND,27.0,HOU,29.0,23.8,31.8,0,0
7,MIA,20.0,JAX,17.0,24.9,17.7,1,1
8,NO,47.0,CAR,10.0,32.9,8.4,1,1
9,NYG,6.0,MIN,28.0,13.3,26.6,0,0


from matplotlib import pyplot as plt
_df_0['home_score'].plot(kind='hist', bins=20, title='home_score')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['away_score'].plot(kind='hist', bins=20, title='away_score')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['home_score_pred'].plot(kind='hist', bins=20, title='home_score_pred')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_3['away_score_pred'].plot(kind='hist', bins=20, title='away_score_pred')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
_df_7.plot(kind='scatter', x='away_score_pred', y='home_team_win', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_8['home_score'].plot(kind='line', figsize=(8, 4), title='home_score')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_9['away_score'].plot(kind='line', figsize=(8, 4), title='away_score')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_10['home_score_pred'].plot(kind='line', figsize=(8, 4), title='home_score_pred')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_11['away_score_pred'].plot(kind='line', figsize=(8, 4), title='away_score_pred')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
from sklearn.metrics import mean_absolute_error

withoutnan = predictions.dropna()
error = mean_absolute_error(withoutnan['home_score'],withoutnan['home_score_pred'])
error

6.042857142857143

In [ ]:
predictions['error_home'] = predictions.apply(lambda x: abs(x.home_score - x.home_score_pred), axis=1)
predictions['error_away'] = predictions.apply(lambda x: abs(x.away_score - x.away_score_pred), axis=1)
round(((predictions['error_home'].sum()/predictions.shape[0])+(predictions['error_away'].sum()/predictions.shape[0]))/2,1)

6.3